In [5]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("precision",16)
%matplotlib inline

In [108]:
class LSTM_Config(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.lr = 0.0000001
        self.num_layers = 1
        self.hidden_size = 100
        self.input_size = 1
        self.output_size = 1
        self.num_epochs = 1000
        self.batch_size = 50
        self.step_size = 5
        self.reuse = False

In [88]:
class LSTM_Model(object):
    def __init__(self,scope,feed,config):
        self.config = config
        with tf.variable_scope(scope):
            self.feed = feed
            self.inputs = feed.input_data
            self.targets = feed.target_data
            self._build_graph()
    
    def _build_graph(self):
        #self._init_placeholders()
        self._build_model()
        self._setup_training()
        
    def _init_placeholders(self):
        self.input_ph = tf.placeholder(
            tf.float32, 
            [None, self.config.step_size, self.config.input_size],
            name="input_ph"
        )
        self.labels_ph = tf.placeholder(
            tf.float32,
            [None, self.config.step_size, self.config.output_size],
            name="labels_ph"
        )
        
    def _build_model(self):
        with tf.variable_scope("model", reuse=self.config.reuse):
            lstm_cell = tf.contrib.rnn.LSTMCell(
                self.config.hidden_size,
                activation=tf.nn.relu,
                state_is_tuple=True
            )
            
            batch_size = tf.shape(self.inputs)[0]
            step_size = tf.shape(self.inputs)[1]
            
            self.state_in = lstm_cell.zero_state(batch_size, tf.float32)
            
            self.model, self.state = tf.nn.dynamic_rnn(
                lstm_cell,
                self.inputs,
                initial_state=self.state_in,
                time_major=False
            )
            
            c,h = self.state
            self.state_out = (c,h)
            self.model = tf.reshape(self.model, shape=[-1,self.config.hidden_size])
            
            weights = tf.Variable(tf.random_uniform(
                [self.config.hidden_size, self.config.output_size],
                minval=-0.05, maxval=0.05
            ))
            biases = tf.Variable(tf.random_uniform(
                [self.config.output_size],
                minval=-0.05, maxval=0.05
            ))
            self.model = tf.nn.xw_plus_b(self.model, weights, biases, name="model")
            self.model = tf.reshape(self.model, shape=[batch_size, step_size, self.config.output_size])
    
    def _setup_training(self):
        self.loss = tf.reduce_sum(tf.square(self.model - self.targets))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.config.lr)
        self.training = self.optimizer.minimize(self.loss)

In [98]:
from tensorflow.contrib.data.python.ops import sliding

def batch_producer(raw_data, raw_targets, batch_size, step_size):
    raw_data_tensor = tf.convert_to_tensor(raw_data, name="raw_data", dtype=tf.float32)
    raw_targets_tensor = tf.convert_to_tensor(raw_targets, name="raw_targets", dtype=tf.float32)
    
    data_len = tf.shape(raw_data_tensor)[0]
    batched_len = data_len // batch_size
    data = tf.reshape(raw_data_tensor[0:batch_size*batched_len],
                     [batch_size, batched_len])
    targets = tf.reshape(raw_targets_tensor[0:batch_size*batched_len],
                        [batch_size, batched_len])

    data = tf.data.Dataset.from_tensor_slices(tf.transpose(data))
    targets = tf.data.Dataset.from_tensor_slices(tf.transpose(targets))
    window = step_size
    stride = 1

    data = data.apply(sliding.sliding_window_batch(window,stride))
    targets = targets.apply(sliding.sliding_window_batch(window,stride))

    data_iterator = tf.data.Iterator.from_structure(data.output_types, data.output_shapes)
    target_iterator = tf.data.Iterator.from_structure(targets.output_types, targets.output_shapes)
    next_element = data_iterator.get_next()
    target_element = target_iterator.get_next()

    next_batch = tf.expand_dims(next_element,0)
    next_batch = tf.transpose(next_batch)
    next_targets = tf.expand_dims(target_element,0)
    next_targets = tf.transpose(next_targets)

    init_op = [data_iterator.make_initializer(data),target_iterator.make_initializer(targets)]
    return next_batch, next_targets, init_op
    
class Data_Feed(object):
    def __init__(self, data, targets, batch_size, step_size):
        self.batch_size = batch_size
        self.step_size = step_size
        #print(len(data) // batch_size)
        self.epoch_size = ((len(data) // batch_size)-1) // step_size
        print(self.epoch_size)
        self.input_data, self.target_data, self.init_op = \
            batch_producer(data,targets, batch_size, step_size)

In [104]:
def train(training_data, training_targets, config):
    
    tf.reset_default_graph()
    
    training_feed = Data_Feed(training_data, training_targets, config.batch_size, config.step_size)
    model = LSTM_Model('model', training_feed, config)
    
    
    print_interval = 1
    init_op = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        
        sess.run([init_op])
        sess.run([training_feed.init_op])
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        saver = tf.train.Saver()
        
        for epoch in range(config.num_epochs):
            lstm_state = np.zeros((2, config.batch_size, config.hidden_size))
            
            try:
                for step in range(training_feed.epoch_size):
                    if step % print_interval != 0:
                        loss, _,  lstm_state = sess.run(\
                            [model.loss, model.training, model.state],
                            feed_dict={model.state_in[0] : lstm_state[0],
                                      model.state_in[1] : lstm_state[1]})
                    else:
                        loss, _,  lstm_state = sess.run(\
                            [model.loss, model.training, model.state],
                            feed_dict={model.state_in[0] : lstm_state[0],
                                      model.state_in[1] : lstm_state[1]})
                        print("Epoch {}, Step {}, Loss : {}".format(
                            epoch, step, loss
                        ))
            except tf.errors.OutOfRangeError:
                print("End of epoch...")
                sess.run([training_feed.init_op])
        coord.request_stop()
        coord.join(threads)

In [86]:
%%time
data = pd.read_csv("./train/train.csv", dtype={'acoustic_data':np.int16, 'time_to_failure':np.float32})

CPU times: user 2min 54s, sys: 28.6 s, total: 3min 23s
Wall time: 3min 24s


In [94]:
signal_train, fail_train = data['acoustic_data'].values[41:100041], \
                            data['time_to_failure'].values[41:100041]
signal_test, fail_test = data['acoustic_data'].values[100041:120041], \
                            data['time_to_failure'].values[100041:120041]

print(len(signal_train))

100000


In [81]:
#del data

In [109]:
config =  LSTM_Config()
#feed = Data_Feed(signal_train, fail_train, config.batch_size, config.step_size)
ct = 0

"""
with tf.Session() as sess:
    sess.run(feed_test.init_op)
    while True:
        try:
            val = sess.run(feed_test.input_data)
            print(val)
            print(ct)
            ct += 1
        except:
            print("Failed")
            break
"""
train(signal_train, fail_train, config)

399
Epoch 0, Step 0, Loss : 507.79638671875
Epoch 0, Step 1, Loss : 499.79827880859375
Epoch 0, Step 2, Loss : 500.9070739746094
Epoch 0, Step 3, Loss : 503.0704345703125
Epoch 0, Step 4, Loss : 505.3995666503906
Epoch 0, Step 5, Loss : 510.5587158203125
Epoch 0, Step 6, Loss : 524.435302734375
Epoch 0, Step 7, Loss : 529.12451171875
Epoch 0, Step 8, Loss : 538.4261474609375
Epoch 0, Step 9, Loss : 862.252685546875
Epoch 0, Step 10, Loss : 1242.7371826171875
Epoch 0, Step 11, Loss : 700.935546875
Epoch 0, Step 12, Loss : 768.528564453125
Epoch 0, Step 13, Loss : 1030.80908203125
Epoch 0, Step 14, Loss : 1506.082275390625
Epoch 0, Step 15, Loss : 2107.952880859375
Epoch 0, Step 16, Loss : 4606.67724609375
Epoch 0, Step 17, Loss : 6082.43603515625
Epoch 0, Step 18, Loss : 15231.244140625
Epoch 0, Step 19, Loss : 32049.78125
Epoch 0, Step 20, Loss : 72505.4765625
Epoch 0, Step 21, Loss : 235303.703125
Epoch 0, Step 22, Loss : 624580.75
Epoch 0, Step 23, Loss : 2084209.125
Epoch 0, Step 24

KeyboardInterrupt: 

In [11]:
del data

In [103]:
test = signal_train[0:20]
y = fail_train[0:20]
batch_size = 3
step_size = 2

data_len = len(test)
batched_len = data_len // batch_size
epoch_size = (batched_len-1) // step_size

training_data = np.reshape(test[0:batched_len*batch_size],
                          [batch_size, batched_len])
training_label = np.reshape(y[0:batched_len*batch_size],
                           [batch_size, batched_len])



print(test)
print(training_data)
print(np.shape(training_data))
print(training_label)

[ 3  6  3  4  2  3 10  6  8 11 11 11  2  9  7  3  8  7  4  4]
[[ 3  6  3  4  2  3]
 [10  6  8 11 11 11]
 [ 2  9  7  3  8  7]]
(3, 6)
[[1.4690999 1.4690999 1.4690999 1.4690999 1.4690999 1.4690999]
 [1.4690999 1.4690999 1.4690999 1.4690999 1.4690999 1.4690999]
 [1.4690999 1.4690999 1.4690999 1.4690999 1.4690999 1.4690999]]


In [63]:
test = signal_train[0:20]
num_periods = 20
x_data = test[0:(len(test)-(len(test) % num_periods))]
x_batches = x_data.reshape(-1,20,1)
print(len(test) % num_periods)
print(x_batches.shape)
print(x_batches)

0
(1, 20, 1)
[[[ 3]
  [ 6]
  [ 3]
  [ 4]
  [ 2]
  [ 3]
  [10]
  [ 6]
  [ 8]
  [11]
  [11]
  [11]
  [ 2]
  [ 9]
  [ 7]
  [ 3]
  [ 8]
  [ 7]
  [ 4]
  [ 4]]]


In [117]:
from tensorflow.contrib.data.python.ops import sliding

data = tf.data.Dataset.from_tensor_slices(training_data.T)
targets = tf.data.Dataset.from_tensor_slices(training_label.T)
window = 2
stride = 1

data = data.apply(sliding.sliding_window_batch(window,stride))
targets = targets.apply(sliding.sliding_window_batch(window,stride))

data_iterator = tf.data.Iterator.from_structure(data.output_types, data.output_shapes)
target_iterator = tf.data.Iterator.from_structure(targets.output_types, targets.output_shapes)
next_element = data_iterator.get_next()
target_element = target_iterator.get_next()

next_batch = tf.expand_dims(next_element,0)
next_batch = tf.transpose(next_batch)
next_targets = tf.expand_dims(target_element,0)
next_targets = tf.transpose(next_targets)

init_op = [data_iterator.make_initializer(data),target_iterator.make_initializer(targets)]

with tf.Session() as sess:
    sess.run(init_op)
    while True:
        try:
            batch = sess.run(next_batch)
            tar = sess.run(next_targets)
            print(batch)
            print(tar)
            print(np.shape(batch))
        except tf.errors.OutOfRangeError:
            print("End of dataset")
            break

AttributeError: 'TensorSliceDataset' object has no attribute 'window'